<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Analisis_Datos_Puntos_Fijos_XLSX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paso 1: Carga de datos de Puntos Fijos desde archivo Excel (.xlsx)

Este script permite cargar los datos de puntos fijos desde un archivo Excel (.xlsx) utilizando varias fuentes posibles:

1. **Desde tu PC:** Sube el archivo manualmente.
2. **Desde Google Drive:** Especifica la ruta en tu Drive.
3. **Desde la carpeta local de Colab (`/content`):** Si el archivo ya está en el entorno de Colab.
4. **Desde un repositorio público de GitHub:** Mediante una URL directa al archivo `.xlsx`.

## Instrucciones

- Ejecuta el código y selecciona el origen de tus datos en el widget interactivo.
- Si seleccionas "PC", se abrirá un cuadro de diálogo para subir tu archivo.
- Si seleccionas "Drive", asegúrate de montar tu Google Drive y especificar la ruta al archivo.
- Si seleccionas "Local", asegúrate de que el archivo esté en `/content` (puedes subirlo previamente).
- Si seleccionas "GitHub", coloca la URL directa al archivo Excel en un repositorio público.

El DataFrame cargado se muestra en pantalla junto con información básica del archivo.

In [19]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# Widget para seleccionar origen
origen_widget = widgets.Dropdown(
    options=['PC', 'Drive', 'Local', 'GitHub'],
    value='PC',
    description='Origen del archivo:',
    style={'description_width': 'initial'}
)

ruta_drive_widget = widgets.Text(
    value='',
    placeholder='Escribe la ruta completa en tu Google Drive',
    description='Ruta Drive:',
    disabled=False
)

ruta_local_widget = widgets.Text(
    value='',
    placeholder='Escribe el nombre del archivo en /content',
    description='Archivo Local:',
    disabled=False
)

url_github_widget = widgets.Text(
    value='',
    placeholder='Pega la URL directa al archivo .xlsx en GitHub',
    description='URL GitHub:',
    disabled=False
)

output = widgets.Output()

def cargar_datos(origen):
    df_local = None
    nombre_archivo = ""
    if origen == 'PC':
        from google.colab import files
        uploaded = files.upload()
        if not uploaded:
            print("No se seleccionó ningún archivo.")
            return
        nombre_archivo = list(uploaded.keys())[0]
        if not nombre_archivo.lower().endswith('.xlsx'):
            print("Por favor, sube un archivo .xlsx válido.")
            return
        df_local = pd.read_excel(nombre_archivo)
    elif origen == 'Drive':
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        ruta = ruta_drive_widget.value
        if not ruta or not os.path.exists(ruta):
            print("Especifica una ruta válida en tu Google Drive.")
            return
        nombre_archivo = os.path.basename(ruta)
        df_local = pd.read_excel(ruta)
    elif origen == 'Local':
        ruta = ruta_local_widget.value
        if not ruta or not os.path.exists(ruta):
            print("Especifica un archivo válido en /content.")
            return
        nombre_archivo = os.path.basename(ruta)
        df_local = pd.read_excel(ruta)
    elif origen == 'GitHub':
        url = url_github_widget.value
        if not url.lower().endswith('.xlsx'):
            print("Debes proporcionar una URL directa a un archivo .xlsx en GitHub.")
            return
        nombre_archivo = url.split("/")[-1]
        df_local = pd.read_excel(url)
    else:
        print("Origen no válido.")
        return

    # Hacer que 'df' esté disponible globalmente
    globals()['df'] = df_local

    print(f"Archivo cargado: {nombre_archivo}")
    print(df_local.shape)
    display(df_local.head())
    return df_local

def mostrar_widgets_de_origen(change):
    clear_output(wait=True)
    display(origen_widget)
    if origen_widget.value == 'Drive':
        display(ruta_drive_widget)
    elif origen_widget.value == 'Local':
        display(ruta_local_widget)
    elif origen_widget.value == 'GitHub':
        display(url_github_widget)
    display(boton_cargar)
    display(output)

def boton_cargar_click(b):
    output.clear_output()
    with output:
        cargar_datos(origen_widget.value)

# Botón para ejecutar la carga
boton_cargar = widgets.Button(description="Cargar archivo")
boton_cargar.on_click(boton_cargar_click)

origen_widget.observe(mostrar_widgets_de_origen, names='value')

# Mostrar widgets iniciales
display(origen_widget)
display(boton_cargar)
display(output)
mostrar_widgets_de_origen(None)

Dropdown(description='Origen del archivo:', options=('PC', 'Drive', 'Local', 'GitHub'), style=DescriptionStyle…

Button(description='Cargar archivo', style=ButtonStyle())

Output()

# Paso 2: Limpieza y preprocesamiento de datos (archivo Excel)

Este paso toma el DataFrame cargado desde el archivo Excel (.xlsx) y realiza una limpieza y preprocesamiento básico para preparar los datos para el análisis:

1. **Uniformiza los nombres de las columnas:**  
   Convierte todos los nombres de las columnas a mayúsculas y reemplaza los espacios por guiones bajos (`_`).

2. **Convierte la columna de fecha a tipo datetime:**  
   Busca automáticamente la columna que contiene "FECHA" en su nombre y la convierte a tipo fecha.

3. **Elimina filas vacías y duplicadas:**  
   Borra filas completamente vacías y registros duplicados.

4. **Convierte columnas numéricas:**  
   Intenta convertir todas las columnas (excepto la de fecha) a tipo numérico cuando sea posible.

5. **Muestra el resultado:**  
   Presenta el tamaño del DataFrame resultante, una vista previa, los tipos de datos y el conteo de valores nulos por columna.

> Al final de este paso tendrás tus datos uniformizados, limpios y listos para el análisis exploratorio.

In [2]:
import pandas as pd
import numpy as np

try:
    df
except NameError:
    print("❌ La variable 'df' no está definida. Por favor, ejecuta primero la celda de carga de datos y asegúrate de que el DataFrame se llame 'df'.")
else:
    # --- 1. Uniformizar nombres de columnas (mayúsculas y _ en vez de espacios) ---
    df.columns = [col.strip().upper().replace(" ", "_") for col in df.columns]

    # --- 2. Detectar y convertir columnas de fechas ---
    columna_fecha = None
    for col in df.columns:
        if "FECHA" in col:
            columna_fecha = col
            break

    if columna_fecha:
        df[columna_fecha] = pd.to_datetime(df[columna_fecha], errors='coerce', dayfirst=True)

    # --- 3. Eliminar filas completamente vacías y duplicadas ---
    df = df.dropna(how='all')
    df = df.drop_duplicates()

    # --- 4. Convertir columnas numéricas a tipo numérico cuando sea posible ---
    for col in df.columns:
        if col != columna_fecha:
            df[col] = pd.to_numeric(df[col], errors='ignore')

    # --- 5. Vista rápida del resultado ---
    print("Shape luego de limpieza:", df.shape)
    display(df.head())
    print(df.dtypes)
    print(df.isnull().sum())

❌ La variable 'df' no está definida. Por favor, ejecuta primero la celda de carga de datos y asegúrate de que el DataFrame se llame 'df'.


# Paso 3: Análisis exploratorio de datos (EDA) - Archivo Excel

En este paso se realiza un Análisis Exploratorio de Datos (EDA) sobre el DataFrame limpio proveniente del archivo Excel. El objetivo es obtener una visión general de los datos, detectar patrones, distribuciones, valores atípicos y posibles problemas antes de avanzar al análisis estadístico o modelado.

## Incluye:

1. **Resumen general del DataFrame**  
   - Número de filas y columnas.
   - Tipos de datos por columna.
   - Conteo de valores nulos por columna.

2. **Estadísticas descriptivas**
   - Estadísticas básicas de las columnas numéricas: media, desviación estándar, mínimo, máximo, percentiles, etc.
   - Estadísticas de columnas categóricas: conteo de valores únicos y frecuencia de las categorías principales.

3. **Visualización rápida**
   - Histogramas para variables numéricas.
   - Gráficos de barras para variables categóricas relevantes.
   - Boxplots para detectar valores atípicos en variables numéricas principales.

> Este paso es fundamental para familiarizarte con la estructura y calidad de los datos, y para definir estrategias de análisis posteriores.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Asegúrate de que el DataFrame limpio se llame "df"

try:
    df
except NameError:
    print("❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga y limpieza de datos.")
else:
    print("=== Resumen general ===")
    print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")
    print("\nTipos de datos:")
    print(df.dtypes)
    print("\nValores nulos por columna:")
    print(df.isnull().sum())

    print("\n=== Estadísticas descriptivas (numéricas) ===")
    display(df.describe())

    # Estadísticas para variables categóricas
    print("\n=== Estadísticas descriptivas (categóricas) ===")
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        print(f"\nColumna: {col}")
        print(df[col].value_counts(dropna=False).head())

    # Visualización rápida
    print("\n=== Visualización rápida ===")
    # Histogramas para variables numéricas
    num_cols = df.select_dtypes(include=['number']).columns
    if len(num_cols) > 0:
        df[num_cols].hist(bins=20, figsize=(15, 6))
        plt.suptitle('Histogramas de variables numéricas')
        plt.show()

    # Gráficos de barras para variables categóricas principales
    for col in cat_cols:
        if df[col].nunique() < 20:
            plt.figure(figsize=(8, 4))
            sns.countplot(data=df, y=col, order=df[col].value_counts().index)
            plt.title(f'Frecuencia de {col}')
            plt.show()

    # Boxplots para variables numéricas (opcional)
    for col in num_cols:
        plt.figure(figsize=(6, 1.5))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot de {col}')
        plt.show()

❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga y limpieza de datos.


# Paso 4: Validación y control de calidad de los datos (archivo Excel)

Este paso se encarga de validar y controlar la calidad de los datos del DataFrame previamente limpiado y explorado. El objetivo es detectar problemas que puedan afectar el análisis, como valores atípicos, inconsistencias, formatos erróneos o datos faltantes críticos.

## Incluye:

1. **Valores faltantes**
   - Identifica columnas con porcentaje alto de valores nulos.
   - Muestra filas con datos críticos faltantes (por ejemplo, coordenadas, fechas, identificadores).

2. **Duplicados y unicidad**
   - Revisa si existen duplicados en identificadores clave (por ejemplo, ID, nombre del punto, combinación de columnas relevantes).

3. **Consistencia de formatos**
   - Verifica formatos de fechas y columnas numéricas.
   - Valida rangos plausibles para latitud, longitud y otros campos importantes.

4. **Valores atípicos**
   - Detecta outliers en variables numéricas principales mediante boxplots y z-score.

5. **Reporte de advertencias**
   - Genera advertencias sobre problemas detectados y sugiere acciones de corrección.

> Este validador ayuda a asegurar la confiabilidad de los datos antes de cualquier análisis avanzado, modelado o visualización.

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Asegúrate de que el DataFrame limpio se llame "df"
try:
    df
except NameError:
    print("❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga, limpieza y EDA de datos.")
else:
    print("=== VALIDADOR DE CALIDAD DE DATOS ===")

    # 1. Valores faltantes
    print("\n--- Valores faltantes por columna:")
    faltantes = df.isnull().mean() * 100
    print(faltantes[faltantes > 0].sort_values(ascending=False))

    # Ejemplo: mostrar filas con datos críticos faltantes (ajusta el nombre según tu caso)
    columnas_criticas = []
    for clave in ['LAT', 'LON', 'LONG', 'FECHA', 'ID']:
        columnas_criticas.extend([col for col in df.columns if clave in col])
    columnas_criticas = list(set(columnas_criticas))
    if columnas_criticas:
        print(f"\nFilas con datos críticos faltantes en: {columnas_criticas}")
        display(df[df[columnas_criticas].isnull().any(axis=1)].head())
    else:
        print("\nNo se detectaron columnas críticas predefinidas para validación de nulos.")

    # 2. Duplicados y unicidad
    if 'ID' in df.columns:
        duplicados = df.duplicated(subset=['ID'])
        print(f"\nFilas duplicadas por 'ID': {duplicados.sum()}")
        if duplicados.sum() > 0:
            display(df[duplicados])
    else:
        print("\nNo se encontró columna 'ID' para chequeo de duplicados.")

    # 3. Consistencia de formatos y rangos plausibles
    for col in df.columns:
        if "FECHA" in col:
            n_fechas_invalidas = df[col].isnull().sum()
            print(f"\nFechas inválidas en '{col}': {n_fechas_invalidas}")
        if "LAT" in col:
            lat_validas = df[col].between(-90, 90)
            print(f"Latitudes fuera de rango en '{col}': {(~lat_validas).sum()}")
        if "LON" in col or "LONG" in col:
            lon_validas = df[col].between(-180, 180)
            print(f"Longitudes fuera de rango en '{col}': {(~lon_validas).sum()}")

    # 4. Valores atípicos (outliers) en numéricas
    print("\n--- Búsqueda de outliers (z-score > 3 o < -3):")
    num_cols = df.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        zs = zscore(df[col].dropna())
        outliers = np.where(np.abs(zs) > 3)[0]
        print(f"{col}: {len(outliers)} posibles outliers")

    print("\n--- Validación completada. Revisa advertencias anteriores para posibles acciones correctivas.")

❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga, limpieza y EDA de datos.


# Paso 5: Análisis estadístico avanzado (archivo Excel)

Este paso realiza un análisis estadístico más profundo sobre el DataFrame limpio. El objetivo es identificar relaciones, correlaciones, tendencias y comportamientos significativos en los datos, y sentar las bases para la modelización o la toma de decisiones.

## Incluye:

1. **Análisis de correlación**
   - Matriz de correlación entre variables numéricas.
   - Visualización con mapa de calor.

2. **Tablas de contingencia**
   - Frecuencias cruzadas entre variables categóricas seleccionadas.

3. **Comparaciones de grupos**
   - Estadísticas descriptivas agrupadas por una variable categórica relevante (si la hay).

4. **Pruebas estadísticas básicas**
   - Prueba de normalidad para variables numéricas principales.
   - Prueba de diferencia de medias entre grupos (t-test o ANOVA), si corresponde.

> Este paso permite detectar patrones y relaciones que no son evidentes a simple vista y sirve de base para análisis predictivos o de segmentación.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Asegúrate de que el DataFrame limpio se llame "df"
try:
    df
except NameError:
    print("❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga y limpieza de datos.")
else:
    print("=== ANÁLISIS ESTADÍSTICO AVANZADO ===\n")

    # 1. Matriz de correlación numérica
    num_cols = df.select_dtypes(include=[np.number]).columns
    if len(num_cols) > 1:
        print("Matriz de correlación:")
        corr = df[num_cols].corr()
        display(corr)
        plt.figure(figsize=(8, 6))
        sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title("Mapa de calor de correlación")
        plt.show()
    else:
        print("No hay suficientes columnas numéricas para correlación.")

    # 2. Tablas de contingencia (para categóricas)
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(cat_cols) >= 2:
        print("\nTablas de contingencia entre variables categóricas principales:")
        for i in range(min(2, len(cat_cols))):
            for j in range(i+1, len(cat_cols)):
                print(f"\nTabla de {cat_cols[i]} vs {cat_cols[j]}:")
                display(pd.crosstab(df[cat_cols[i]], df[cat_cols[j]]))
    else:
        print("No hay suficientes columnas categóricas para tablas de contingencia.")

    # 3. Estadísticas por grupo (si hay variable categórica relevante)
    if len(cat_cols) > 0 and len(num_cols) > 0:
        col_cat = cat_cols[0]
        print(f"\nEstadísticas descriptivas agrupadas por '{col_cat}':")
        display(df.groupby(col_cat)[num_cols].describe())
    else:
        print("No hay suficientes columnas para agrupamientos.")

    # 4. Pruebas estadísticas básicas
    print("\nPrueba de normalidad (Shapiro-Wilk):")
    for col in num_cols:
        vals = df[col].dropna()
        if len(vals) >= 3 and len(vals) <= 5000:  # Shapiro limita n<=5000
            stat, p = stats.shapiro(vals)
            print(f"{col}: p-valor={p:.4f} {'(normal)' if p>0.05 else '(no normal)'}")
        elif len(vals) > 5000:
            print(f"{col}: Demasiados datos para Shapiro (n={len(vals)}), usar KS-test.")
            stat, p = stats.kstest((vals - vals.mean())/vals.std(ddof=0), 'norm')
            print(f"    KS-test: p-valor={p:.4f} {'(normal)' if p>0.05 else '(no normal)'}")
        else:
            print(f"{col}: Insuficientes datos para test de normalidad.")

    # t-test o ANOVA para diferencia de medias si relevante
    if len(cat_cols) > 0 and len(num_cols) > 0:
        col_cat = cat_cols[0]
        for col in num_cols:
            grupos = df[col_cat].dropna().unique()
            if len(grupos) == 2:
                grupo1 = df[df[col_cat]==grupos[0]][col].dropna()
                grupo2 = df[df[col_cat]==grupos[1]][col].dropna()
                if len(grupo1)>1 and len(grupo2)>1:
                    stat, p = stats.ttest_ind(grupo1, grupo2, equal_var=False)
                    print(f"\nT-test {col} entre {grupos[0]} y {grupos[1]}: p-valor={p:.4f} {'(diferencia significativa)' if p<0.05 else '(no significativa)'}")
            elif len(grupos) > 2:
                muestras = [df[df[col_cat]==g][col].dropna() for g in grupos if len(df[df[col_cat]==g][col].dropna())>1]
                if len(muestras) >= 2:
                    stat, p = stats.f_oneway(*muestras)
                    print(f"\nANOVA {col} para grupos en {col_cat}: p-valor={p:.4f} {'(diferencia significativa)' if p<0.05 else '(no significativa)'}")
    print("\n--- Fin del análisis estadístico avanzado ---")

❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga y limpieza de datos.


# Paso 6: Visualización avanzada de datos (archivo Excel)

En este paso se generan visualizaciones más sofisticadas e interactivas para profundizar en la comprensión de los datos y comunicar hallazgos de manera efectiva.

## Incluye:

1. **Gráficos interactivos**
   - Uso de bibliotecas como Plotly, Altair o ipywidgets para explorar datos dinámicamente.
   - Histogramas, cajas, gráficos de dispersión y de barras interactivos.

2. **Dashboards básicos**
   - Creación de un tablero simple con varias visualizaciones relevantes para el análisis.

3. **Visualización de relaciones**
   - Diagramas de dispersión, pares de variables, diagramas de violín, etc.
   - Visualización de correlaciones y tendencias.

4. **Personalización y exportación**
   - Personalización de títulos, etiquetas y colores.
   - Exportación de gráficos a archivos de imagen o HTML interactivo.

> Este paso es ideal para presentaciones, análisis exploratorios en profundidad y para identificar patrones que no se observan fácilmente en tablas o gráficos estáticos.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Opcional: plotly express para gráficos interactivos
try:
    import plotly.express as px
    PLOTLY = True
except ImportError:
    PLOTLY = False

try:
    df
except NameError:
    print("❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga y limpieza de datos.")
else:
    print("=== VISUALIZACIÓN AVANZADA DE DATOS ===\n")

    # Histogramas y boxplots para numéricas
    num_cols = df.select_dtypes(include='number').columns
    if len(num_cols) > 0:
        for col in num_cols:
            fig, axes = plt.subplots(1, 2, figsize=(12, 4))
            sns.histplot(df[col].dropna(), kde=True, ax=axes[0])
            axes[0].set_title(f"Histograma de {col}")
            sns.boxplot(x=df[col], ax=axes[1])
            axes[1].set_title(f"Boxplot de {col}")
            plt.tight_layout()
            plt.show()

    # Gráficos de barras para categóricas principales
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    for col in cat_cols:
        if df[col].nunique() < 25:
            plt.figure(figsize=(8, 4))
            sns.countplot(data=df, y=col, order=df[col].value_counts().index)
            plt.title(f"Frecuencia de {col}")
            plt.show()

    # Gráficos de dispersión entre variables numéricas (matriz de pares)
    if len(num_cols) > 1:
        sns.pairplot(df[num_cols].dropna())
        plt.suptitle("Matriz de gráficos de dispersión", y=1.02)
        plt.show()

    # Visualización interactiva opcional con Plotly
    if PLOTLY and len(num_cols) > 1:
        print("Gráfico de dispersión interactivo con Plotly:")
        fig = px.scatter_matrix(df, dimensions=num_cols)
        fig.update_traces(diagonal_visible=False)
        fig.show()

    print("\n--- Fin de la visualización avanzada ---")

❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga y limpieza de datos.


# Paso 7: Exportación y preparación de datos limpios (archivo Excel)

En este paso se realiza la exportación de los datos procesados y limpios para su uso en otros sistemas, herramientas de análisis, o para resguardo. También se pueden preparar subconjuntos o transformaciones específicas del DataFrame según necesidades del análisis.

## Incluye:

1. **Exportación a formatos comunes**
   - Guardar el DataFrame limpio en archivos XLSX, CSV y/o Parquet.
   - Opción de exportar subconjuntos (por ejemplo, solo columnas seleccionadas o solo filas con ciertos criterios).

2. **Transformaciones adicionales**
   - Conversión de tipos de datos si es necesario (fechas, categóricas, etc.).
   - Creación de columnas derivadas útiles para análisis posteriores.

3. **Resumen del proceso**
   - Registro de cambios principales realizados al set de datos.
   - Notas sobre supuestos o decisiones tomadas en la limpieza.

> Este paso asegura que los datos finales sean reutilizables, auditables y compatibles con tus próximos procesos de análisis o presentación.

In [9]:
%pip install ipywidgets plotly
from google.colab import output as colab_output
colab_output.enable_custom_widget_manager()

In [17]:
%pip install -U kaleido

In [38]:
import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

try:
    df
except NameError:
    print("❌ La variable 'df' no está definida. Por favor, ejecuta primero la carga de datos.")
else:
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN']
    variables = [col for col in df.select_dtypes(include='number').columns if col not in columnas_excluidas]
    puntos_fijos = sorted(df['INSTRUMENTO'].dropna().unique())
    opciones_puntos = ["Todos"] + list(puntos_fijos)

    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    opciones_anios = ["Todos"] + [str(a) for a in anios]

    estilos_grafico = [
        "Curvas suaves (spline)",
        "Líneas rectas",
        "Puntos",
        "Líneas + Puntos",
        "Área apilada",
        "Área + Líneas",
        "Área + Líneas + Puntos"
    ]

    tamanios_imagen = {
        "Pequeño (600x400)": (600, 400),
        "Mediano (900x500)": (900, 500),
        "Grande (1200x700)": (1200, 700),
        "Extra grande (1600x1000)": (1600, 1000)
    }

    grosores = {
        "Fino (1px)": 1,
        "Normal (2px)": 2,
        "Medio (4px)": 4,
        "Grueso (7px)": 7,
        "Extra grueso (10px)": 10
    }

    paletas = {
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    }

    punto_dropdown = widgets.Dropdown(
        options=opciones_puntos,
        value="Todos",
        description="Punto Fijo:"
    )
    variable_dropdown = widgets.Dropdown(options=variables, description="Variable:")
    estilo_dropdown = widgets.Dropdown(
        options=estilos_grafico,
        value="Curvas suaves (spline)",
        description="Estilo gráfica:"
    )
    anio_dropdown = widgets.Dropdown(
        options=opciones_anios,
        value="Todos",
        description="Año:"
    )
    tamanio_dropdown = widgets.Dropdown(
        options=list(tamanios_imagen.keys()),
        value="Mediano (900x500)",
        description="Tamaño:"
    )
    grosor_dropdown = widgets.Dropdown(
        options=list(grosores.keys()),
        value="Normal (2px)",
        description="Grosor línea:"
    )
    paleta_dropdown = widgets.Dropdown(
        options=list(paletas.keys()),
        value="Plotly",
        description="Paleta colores:"
    )
    boton = widgets.Button(description="Graficar", button_style="success")
    output = widgets.Output()

    # ---------- BLOQUE PARA GUARDAR LA GRÁFICA -------------
    formatos = {
        "PNG": ".png",
        "JPEG": ".jpg",
        "SVG": ".svg",
        "PDF": ".pdf",
        "HTML": ".html"
    }

    formato_dropdown = widgets.Dropdown(
        options=list(formatos.keys()),
        value="PNG",
        description="Formato:"
    )
    ruta_text = widgets.Text(
        value="grafica_exportada",
        description="Ruta y nombre:",
        placeholder="ej: ./carpeta/mi_grafica"
    )
    boton_guardar = widgets.Button(
        description="Guardar gráfica",
        button_style="info"
    )
    output_guardar = widgets.Output()

    def guardar_grafica(b=None):
        with output_guardar:
            clear_output(wait=True)
            ext = formatos[formato_dropdown.value]
            ruta_archivo = ruta_text.value
            if not ruta_archivo.lower().endswith(ext):
                ruta_archivo += ext
            if 'fig' not in globals() or not isinstance(fig, go.Figure):
                print("❌ Primero debes generar una gráfica.")
                return
            try:
                if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                    if importlib.util.find_spec("kaleido") is None:
                        print("❌ Para guardar como imagen/vector/pdf, instala 'kaleido':\n%pip install -U kaleido")
                        return
                    fig.write_image(ruta_archivo, format=formato_dropdown.value.lower())
                elif formato_dropdown.value == "HTML":
                    fig.write_html(ruta_archivo)
                else:
                    print("❌ Tipo de archivo no soportado.")
                    return
            except Exception as e:
                print("❌ Error al guardar la gráfica:", e)
                return

            print(f"✅ Gráfica guardada en: {os.path.abspath(ruta_archivo)}")
            if os.path.exists(ruta_archivo):
                if 'google.colab' in str(get_ipython()):
                    from google.colab import files
                    files.download(ruta_archivo)
                else:
                    ruta_abs = os.path.abspath(ruta_archivo)
                    display(HTML(f'<a href="file://{ruta_abs}" target="_blank">Descargar archivo</a>'))

    boton_guardar.on_click(guardar_grafica)
    controles_guardar = widgets.HBox([formato_dropdown, ruta_text, boton_guardar])

    # ---------- FIN BLOQUE GUARDADO ------------------------

    def graficar(b=None):
        global fig
        with output:
            clear_output(wait=True)
            variable = variable_dropdown.value
            estilo = estilo_dropdown.value
            punto = punto_dropdown.value
            anio = anio_dropdown.value
            ancho, alto = tamanios_imagen[tamanio_dropdown.value]
            grosor = grosores[grosor_dropdown.value]
            paleta = paletas[paleta_dropdown.value]

            df_plot = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])

            if anio != "Todos":
                df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
            if punto != "Todos":
                df_plot = df_plot[df_plot['INSTRUMENTO'] == punto]
            if df_plot.empty:
                print("No hay datos para graficar con la selección actual.")
                return

            fig = go.Figure()
            instrumentos = sorted(df_plot['INSTRUMENTO'].unique())
            color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}
            for pf in instrumentos:
                data_pf = df_plot[df_plot['INSTRUMENTO'] == pf]
                line_args = dict(width=grosor, color=color_map[pf])
                marker_args = dict(color=color_map[pf])
                if estilo == "Curvas suaves (spline)":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        line_shape="spline",
                        line=line_args
                    ))
                elif estilo == "Líneas rectas":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        line_shape="linear",
                        line=line_args
                    ))
                elif estilo == "Puntos":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="markers",
                        name=pf,
                        marker=marker_args
                    ))
                elif estilo == "Líneas + Puntos":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines+markers",
                        name=pf,
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))
                elif estilo == "Área apilada":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        stackgroup='one',
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))
                elif estilo == "Área + Líneas":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        fill="tozeroy",
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))
                elif estilo == "Área + Líneas + Puntos":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines+markers",
                        name=pf,
                        fill="tozeroy",
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))

            fig.update_layout(
                width=ancho,
                height=alto,
                title=f"{variable} en función del tiempo por Punto Fijo (PF)",
                xaxis_title="Fecha",
                yaxis_title=variable,
                legend_title="INSTRUMENTO",
                hovermode="x unified"
            )
            fig.show()

    # Selectores arriba, botón abajo
    selectores = widgets.HBox([
        punto_dropdown, variable_dropdown, estilo_dropdown, anio_dropdown,
        tamanio_dropdown, grosor_dropdown, paleta_dropdown
    ])
    controls = widgets.VBox([
        selectores,
        boton
    ])

    display(controls, output, controles_guardar, output_guardar)
    boton.on_click(graficar)

Output()

Output()